
# Load table

In [1]:
import sys

try:
    from google.colab import drive    
    drive.mount('/content/gdrive')
        
    !rm -rf github
    !mkdir github
    !cd github && git clone https://github.com/dr315/Finances.git
    !ls /content/github/Finances/
    !pip install yfinance
    !pip install pdfplumber
    
    sys.path.append('/content/github/Finances')
    root = '/content/gdrive/MyDrive/'
    
except:
    root = 'd:/'

import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format
import datetime as dt
import numpy as np
from FinanceTools import *
from OrdersReader import *    
# ReadOrders('d:/Investing/Notas_Clear', 'operations.csv')
df = pd.read_csv(root + '/Investing/operations.csv')    
# df = pd.read_csv(root + '/Investing/empiricus.csv')

# display(df)

## Formating table

In [2]:
 
df = df.iloc[:, :7]
df.columns=['Codigo','Data', 'Valor', 'Quantidade', 'Tipo', 'Categoria', 'Despesas']
 
#Removing the stored dividends in order to donwload again from internet
df = df[df.Tipo != 'Proventos']

if(df['Valor'].apply(type).eq(str).any()): 
    df['Valor'] = df['Valor'].str.replace(',', '')
    df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
    df['Quantidade'] = pd.to_numeric(df['Quantidade'], errors='coerce')
    df['Despesas'] = pd.to_numeric(df['Despesas'], errors='coerce')
 
#drop empty lines
df=df[df['Data'].astype(bool)].dropna()
# df

if(df['Data'].apply(type).eq(str).any()): 
    df.Data = df.Data.str.replace('-','/')
    df['Data'] = pd.to_datetime(df.Data, format='%Y/%m/%d')
df['Year'] = pd.DatetimeIndex(df['Data']).year
df['Month'] = pd.DatetimeIndex(df['Data']).month_name()

#Sort the table by date and Type and reset index numeration
df.sort_values(by=['Data', 'Tipo'], ascending=[True, True], inplace=True)
df.reset_index(drop=True, inplace=True)

#turn all sell amount negative
df.loc[df.Tipo == 'Venda', ['Quantidade']] *= -1

#Get the oldest order date
startDate = df.iloc[0]['Data']

#Calc Operation Value
df['Total'] = df.Valor * df.Quantidade + df.Despesas


In [3]:
df['acum_qty'] = 0
#Get the complete list of assets
stocks = np.sort(df.Codigo.unique()).tolist()
# Clear operation costs befere 2019
# df.Despesas.update(df.apply(clear2018Cost, axis=1))


## PriceReader

In [4]:
# import concurrent.futures
import time
import threading
start_time = time.time()

prcReader = PriceReader(stocks, startDate)
splReader = SplitsReader(df)
divReader = DividendReader(df)
# divReader = YfinanceReader(df)

def threadExecutor(obj):
    obj.load()

threadList = []
threadList.append(threading.Thread(target=threadExecutor, args=(prcReader,)))
threadList.append(threading.Thread(target=threadExecutor, args=(divReader,)))
threadList.append(threading.Thread(target=threadExecutor, args=(splReader,)))

for td in threadList:
    td.start()

for td in threadList:
    td.join()

print("--- %s seconds ---" % (time.time() - start_time))

h:\Git\Finances\FinanceTools.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace('\.SA','')
--- 21.27789616584778 seconds ---


In [5]:
dfBak = df

## Accumulator

In [6]:

df=dfBak
df['acum_qty'] = 0
df['PM'] = 0

for paper in stocks:
  paperTable = df[(df.Codigo == paper)]
  fromDate = paperTable.iloc[0]['Data']  
  toDate = dt.datetime.today().strftime('%Y-%m-%d')

  divTable = divReader.getPeriod(paper,fromDate, toDate)
  divTable['Quantidade'] = 0
  divTable['Tipo'] = 'Proventos'
  divTable['Categoria'] = paperTable.iloc[0]['Categoria']
  divTable['Despesas'] = 0
  divTable['Year'] = pd.DatetimeIndex(divTable.index).year
  divTable['Month'] = pd.DatetimeIndex(divTable.index).month_name()
  divTable['Total'] = 0
  divTable['acum_qty'] = 0
  df = df.append(divTable.reset_index())

  splitTable = splReader.getPeriod(paper,fromDate, toDate)
  splitTable['Valor'] = 0
  splitTable['Tipo'] = 'Split'
  splitTable['Categoria'] = paperTable.iloc[0]['Categoria']
  splitTable['Despesas'] = 0
  splitTable['Year'] = pd.DatetimeIndex(splitTable.index).year
  splitTable['Month'] = pd.DatetimeIndex(splitTable.index).month_name()
  splitTable['Total'] = 0
  splitTable['acum_qty'] = 0
  df = df.append(splitTable.reset_index())

df.sort_values(['Data', 'Tipo'], inplace=True)

#Create obj
ta = TableAccumulator()
#Calc the average price and rename the columns names
df=df.groupby(['Codigo']).apply(ta.ByGroup).reset_index(drop=True).dropna()
df.sort_values(['Data', 'Tipo'], inplace=True)

In [7]:
# df[df.Codigo.str.contains('CIEL3')]

# Realized Profit


## Processing

In [8]:

profit = Profit()
df.sort_values(by=['Data', 'Tipo'], ascending=[True, True], inplace=True)
df.reset_index(drop=True)
df=df.groupby(['Codigo', 'Data']).apply(profit.Trade).reset_index()
df.drop('index', axis=1, inplace=True)


## View

In [9]:
rl = df[df.Tipo == 'Venda'][['Data',	'Codigo', 'Categoria', 'Profit',	'DayTrade']]
rl['Data'] = rl['Data'].apply(lambda x: x.strftime('%Y-%m-%d'))
rl.loc['Total', 'Profit'] = rl['Profit'].sum()
rl.fillna(' ', inplace=True)
rl.style.applymap(color_negative_red, subset=['Profit']).format( {'Profit': 'R$ {:,.2f}', 'DayTrade': '{}'})


,Data,Codigo,Categoria,Profit,DayTrade
Total,,,,R$ 0.00,


# Investiment Portifolio


In [10]:
Portifolio(prcReader,df).show()

,PM,Cotacao,Quantidade,Custo,Valor,Proventos,Rentabilidade,Lucro,%R,%R+d
Ativo,,,,,,,,,,
B3SA3,R$ 57.15,R$ 55.30,24,"R$ 1,371.58","R$ 1,327.20",R$ 33.42,R$ -44.38,R$ -10.96,-3.24%,-0.80%
BPAC11,R$ 87.84,R$ 98.60,13,"R$ 1,141.87","R$ 1,281.80",R$ 4.23,R$ 139.93,R$ 144.16,12.25%,12.62%
BPAN4,R$ 8.33,R$ 14.03,760,"R$ 6,330.36","R$ 10,662.80",R$ 130.81,"R$ 4,332.44","R$ 4,463.25",68.44%,70.51%
CASH3,R$ 27.21,R$ 32.91,19,R$ 516.99,R$ 625.29,R$ 0.00,R$ 108.30,R$ 108.30,20.95%,20.95%
CIEL3,R$ 3.50,R$ 3.75,50,R$ 174.81,R$ 187.50,R$ 0.00,R$ 12.69,R$ 12.69,7.26%,7.26%
COCE5,R$ 57.48,R$ 57.29,29,"R$ 1,666.89","R$ 1,661.41",R$ 0.00,R$ -5.48,R$ -5.48,-0.33%,-0.33%
CSAN3,R$ 92.88,R$ 89.87,4,R$ 371.51,R$ 359.48,R$ 0.00,R$ -12.03,R$ -12.03,-3.24%,-3.24%
DIRR3,R$ 12.20,R$ 13.00,12,R$ 146.44,R$ 156.00,R$ 0.00,R$ 9.56,R$ 9.56,6.53%,6.53%
ENEV3,R$ 1.48,R$ 16.78,5645,"R$ 8,349.84","R$ 94,723.10",R$ 0.00,"R$ 86,373.26","R$ 86,373.26","1,034.43%","1,034.43%"


# Performance Blueprint

In [11]:

p = PerformanceBlueprint(prcReader, df, dt.datetime.today().strftime('%m-%d-%Y'))
PerformanceViewer(p.calc()).show()


,Value R$,%C
Item,,
Equity,"156,169.47",318.7%
Cost,"49,005.44",100.0%
Expenses,14.70,0.0%
Paper profit,"107,164.03",218.7%
Realized profit,0.00,0.0%
Dividends,"1,243.07",2.5%
Total Profit,"108,407.10",221.2%


# Taxation


In [12]:
profitLossDF = df.loc[df['Profit'] != 0]
tx = Taxation(profitLossDF)

## FII

### Swing Trade

In [13]:
tx.Process('FII')
display(tx.swingTradeTable)

KeyError: 'Year'

### Day Trade

In [ ]:
try:
  display(tx.dayTradeTable)
except:
  print('Not avalilable')

## Stocks


### Swing Trade

In [ ]:
tx.Process('Stock')
display(tx.swingTradeTable)

### Day Trade

In [ ]:
try:
  display(tx.dayTradeTable)
except:
  print('Not avalilable')

# Plots

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
 
stock_picker = widgets.Dropdown(options=stocks, value=stocks[0])
stock_picker

## Average price Evolution

In [ ]:
stock = stock_picker.value

fig, axs = plt.subplots(2, 1, figsize=(16, 9), sharex=True, gridspec_kw={'height_ratios': [4, 1]})
pickedDF = df.loc[df.Codigo == stock]

div = pickedDF.loc[pickedDF.Tipo == 'Proventos']

prc = prcReader.getHistory([stock], pickedDF.iloc[0]['Data'].strftime('%d-%m-%y'))
axs[0].plot(pickedDF.Data, pickedDF.PM, label='PM')
axs[0].plot(prc.index, prc[stock], label='Market')
axs[0].grid(True, which='both')
axs[0].legend()

axs[1].plot(pickedDF.Data, pickedDF.acumProv, label='Proventos acumulados')
axs[1].bar(div.Data, div.Valor, label='Proventos')
axs[1].grid(True, which='both')
axs[1].legend()

fig.suptitle(stock)
plt.show()

## Historical profitability

In [ ]:
startTimePicker = widgets.Dropdown(options=['All', '12 months'], value='12 months')
startTimePicker

In [ ]:
startPlot = startDate
frequency = 'SM'
if (startTimePicker.value == "12 months"):
  startPlot = (dt.datetime.today() - pd.Timedelta(365, unit='d')).strftime('%Y-%m-%d')
  frequency = 'W'

monthList = pd.date_range(start=startPlot, end=dt.datetime.today(), freq=frequency).format(formatter=lambda x: x.strftime('%Y-%m-%d'))
monthList.append(dt.datetime.today().strftime('%Y-%m-%d'))
performanceList = []
for i, month in enumerate(monthList):
  p = PerformanceBlueprint(prcReader, df, month)
  p.calc()
  performanceList.append([p.date, p.equity, p.cost, p.realizedProfit, p.div, p.paperProfit, p.profit, p.profitRate, p.expense, p.ibov, p.sp500])

histProfDF = pd.DataFrame(performanceList, columns=['Date', 'Equity', 'Cost', 'Profit', 'Div', 'paperProfit', 'TotalProfit', '%Profit', 'Expense', '%IBOV', '%SP500'])
histProfDF['Date'] = pd.to_datetime(histProfDF.Date, format='%Y/%m/%d')
# if (startTimePicker.value == "12 months"):
# histProfDF['%IBOV']   = histProfDF['%IBOV'] - histProfDF.iloc[0]['%IBOV']
# histProfDF['%SP500']  = histProfDF['%SP500'] - histProfDF.iloc[0]['%SP500']
# histProfDF['%Profit'] = histProfDF['%Profit'] - histProfDF.iloc[0]['%Profit']
# histProfDF

In [ ]:
import matplotlib.ticker as mtick

width = 2 if frequency == 'W' else 5     # the width of the bars: can also be len(x) sequence
shift = pd.Timedelta(width/2, unit='d')
fig, ax = plt.subplots(2, 1, figsize=(32, 9), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
fig.tight_layout()

ax[0].plot(histProfDF.Date, histProfDF['%IBOV'], label='ibovespa')
ax[0].plot(histProfDF.Date, histProfDF['%SP500'], label='S&P500')
ax[0].plot(histProfDF.Date, histProfDF['%Profit'], label='Wallet')

minTick = min(histProfDF['%IBOV'].min(), histProfDF['%SP500'].min(), histProfDF['%Profit'].min())
maxTick = max(histProfDF['%IBOV'].max(), histProfDF['%SP500'].max(), histProfDF['%Profit'].max())

ax[0].set_yticks(np.arange(minTick, maxTick, 0.03))
ax[0].axhline(y=0, color='k')
ax[0].grid(True, which='both')
ax[0].yaxis.set_major_formatter(mtick.PercentFormatter(1))
ax[0].legend()

barsDf = histProfDF[:-1]
# ax[1].grid(True, which='both')
ax[1].bar(barsDf.Date - shift, barsDf['Equity'], width, label='Equity')
ax[1].bar(barsDf.Date - shift, barsDf['Div']   , width, bottom=barsDf['Equity'], label='Div')
ax[1].bar(barsDf.Date - shift, barsDf['Profit'], width, bottom=barsDf['Div'] + barsDf['Equity'], label='Profit')
ax[1].bar(barsDf.Date + shift, barsDf['Cost']  , width, label='Cost')
ax[1].legend()
ax[1].set_ylabel('R$')

plt.xticks(barsDf['Date'], rotation=90)
plt.title('Historical profitability')
plt.show()

In [ ]:
data = dt.datetime.today().strftime('%Y-%m-%d')
PerformanceViewer(histProfDF, data).show()


# Dividends by month

In [ ]:
m = int(dt.datetime.today().strftime("%m"))
y = int(dt.datetime.today().strftime("%Y"))
m=3

divTable = divReader.df
prov = df[df['Tipo'] == 'Proventos']

divTable = divTable.reset_index()
divTable = divTable[(divTable['Data de Pagamento'].dt.month == m) & (divTable['Data de Pagamento'].dt.year == y)]

divTable= pd.merge(divTable, prov, how='inner', on=['Data', 'Codigo', 'Valor'])

divTable = divTable[['Data de Pagamento', 'Codigo','Total']]
divTable.columns = ['Data', 'Ativo', 'R$']
# divTable = divTable.groupby(['Ativo', 'Data'])['R$'].sum().reset_index()
# display(divTable)
divTable.sort_values('Data', inplace=True)
divTable['Data'] = divTable['Data'].apply(lambda x: x.strftime('%Y-%m-%d'))
divTable.loc['Total', 'R$'] = divTable['R$'].sum()
divTable.fillna(' ', inplace=True)
divTable


In [ ]:
pvt = prov.pivot_table(index='Year', columns='Month', values='Total', margins=True, margins_name='Total', aggfunc='sum', fill_value=0)
sorted_m = sorted(pvt.columns[:-1], key=lambda month: dt.datetime.strptime(month, "%B"))
sorted_m.append(pvt.columns[-1])
pvt.reindex(sorted_m, axis=1)